#### ID 2020

```Which company had the biggest month call decline from March to April 2020? Return the company_id and calls difference for the company with the highest decline.```

In [ ]:
%%sql
WITH total_call_in_march AS (SELECT company_id,
                                    SUM(CASE WHEN call_id IS NOT NULL THEN 1 ELSE 0 END) AS total_calls
                             FROM rc_calls
                                       JOIN rc_users USING (user_id)
                             WHERE date BETWEEN '2020-03-01 00:00:00' AND '2020-03-31 23:59:59'
                             GROUP BY company_id),
     total_call_in_april AS (SELECT company_id,
                                    SUM(CASE WHEN call_id IS NOT NULL THEN 1 ELSE 0 END) AS total_calls
                             FROM rc_calls
                                      JOIN rc_users USING (user_id)
                             WHERE date BETWEEN '2020-04-01 00:00:00' AND '2020-04-30 23:59:59'
                             GROUP BY company_id),
     variance_calculation AS (SELECT m.company_id,
                                     m.total_calls         AS march_total_calls,
                                     a.total_calls                 AS april_total_calls,
                                     a.total_calls - m.total_calls AS variance
                              FROM total_call_in_march AS m
                                       JOIN total_call_in_april AS a
                                            ON m.company_id = a.company_id)
SELECT company_id, march_total_calls, april_total_calls, variance
FROM variance_calculation
WHERE variance = (SELECT MIN(variance)
                  FROM variance_calculation)
ORDER BY variance;

In [ ]:
total_call_in_march = pd.merge(rc_calls, rc_users, on='user_id', how='inner').query(
    'date >= "2020-03-01 00:00:00" & date <= "2020-03-31 23:59:59"').groupby('company_id', as_index=False).agg(
    total_calls=('call_id', 'count'))

total_call_in_april = pd.merge(rc_calls, rc_users, on='user_id', how='inner').query(
    'date >= "2020-04-01 00:00:00" & date <= "2020-04-30 23:59:59"').groupby('company_id', as_index=False).agg(
    total_calls=('call_id', 'count'))
variance_calculation = pd.merge(total_call_in_march, total_call_in_april, on='company_id', suffixes=('_m', '_a'))
variance_calculation['variance'] = variance_calculation['total_calls_a'] - variance_calculation['total_calls_m']
variance_calculation.nsmallest(1, 'variance')[['company_id', 'variance']]

#### ID 2021

```Redfin helps clients to find agents. Each client will have a unique request_id and each request_id has several calls. For each request_id, the first call is an “initial call” and all the following calls are “update calls”.  What's the average call duration for all initial calls?```

In [ ]:
%%sql
WITH cte AS (SELECT request_id,
                    call_duration,
                    RANK() OVER (PARTITION BY request_id ORDER BY created_on) AS rnk
             FROM redfin_call_tracking)
SELECT AVG(call_duration)
FROM cte
WHERE rnk = 1

In [ ]:
df = redfin_call_tracking
df['rnk'] = df.groupby('request_id')['created_on'].rank(method='first', ascending=True)
df.query('rnk == 1')['call_duration'].mean()

#### ID 2022

```Redfin helps clients to find agents. Each client will have a unique request_id and each request_id has several calls. For each request_id, the first call is an “initial call” and all the following calls are “update calls”.  What's the average call duration for all update calls?```

In [ ]:
%%sql
SELECT AVG(call_duration)
FROM (SELECT call_duration,
             DENSE_RANK() OVER (PARTITION BY request_id ORDER BY created_on) AS rnk
      FROM redfin_call_tracking) t1
WHERE rnk > 1

In [ ]:
df = redfin_call_tracking
df['rnk'] = df.sort_values('created_on').groupby('request_id')['created_on'].rank(method='dense')
df.query('rnk > 1')['call_duration'].mean()

#### 2023

```Redfin helps clients to find agents. Each client will have a unique request_id and each request_id has several calls. For each request_id, the first call is an “initial call” and all the following calls are “update calls”.  How many customers have called 3 or more times between 3 PM and 6 PM (initial and update calls combined)?```

In [ ]:
%%sql
WITH total_calls AS (SELECT request_id, COUNT(call_duration) AS cnt
             FROM redfin_call_tracking
             WHERE EXTRACT(HOUR FROM created_on) BETWEEN 15 AND 18
             GROUP BY request_id
             HAVING COUNT(call_duration) >= 3)
SELECT COUNT(request_id)
FROM total_calls

In [ ]:
df = redfin_call_tracking
df[(df['created_on'].dt.hour >= 15) & (df['created_on'].dt.hour <= 18)].groupby('request_id', as_index=False).agg(
    total_cnt=('call_duration', 'count')).query('total_cnt >= 3')['request_id'].count()

#### ID 2025

```Write a query that returns a number of users who are exclusive to only one client. Output the client_id and number of exclusive users.```

In [ ]:
%%sql
WITH distinct_users AS (SELECT user_id, COUNT(DISTINCT client_id)
                        FROM fact_events
                        GROUP BY user_id
                        HAVING COUNT(DISTINCT client_id) = 1)

SELECT client_id, COUNT(DISTINCT fe.user_id)
FROM fact_events fe
         JOIN distinct_users du ON fe.user_id = du.user_id
GROUP BY client_id

In [ ]:
df = fact_events
grouped_users = df.groupby('user_id', as_index=False).agg(cnt=('client_id', 'nunique')).query('cnt == 1')[
    'user_id'].to_list()
df.query('user_id.isin(@grouped_users)').groupby('client_id', as_index=False).agg(cnt_users=('user_id', 'nunique'))

#### ID 2026

```Write a query that returns a list of the bottom 2 companies by mobile usage. Company is defined in the customer_id column. Mobile usage is defined as the number of events registered on a client_id == 'mobile'. Order the result by the number of events ascending. In the case where there are multiple companies tied for the bottom ranks (rank 1 or 2), return all the companies. Output the customer_id and number of events.```

In [ ]:
%%sql
WITH ranked_mobile_events AS (SELECT customer_id,
                    COUNT(1)                              AS events,
                    DENSE_RANK() OVER (ORDER BY COUNT(1)) AS rnk
             FROM fact_events
             WHERE client_id = 'mobile'
             GROUP BY customer_id)
SELECT customer_id, events
FROM ranked_mobile_events
WHERE rnk = 1

In [ ]:
df = fact_events
df.query('client_id == "mobile"').groupby('customer_id', as_index=False).agg(events=('client_id', 'count')).nsmallest(2,
                                                                                                                      'events',
                                                                                                                      keep='all')

#### ID 2027

```Write a query that returns the company (customer id column) with highest number of users that use desktop only.```

In [ ]:
%%sql
SELECT customer_id
FROM (SELECT customer_id,
             RANK() OVER (
                 ORDER BY COUNT(DISTINCT user_id) DESC) AS rnk
      FROM fact_events
      WHERE user_id IN (SELECT user_id
                        FROM fact_events
                        GROUP BY user_id
                        HAVING COUNT(DISTINCT client_id) = 1)
        AND client_id = 'desktop'
      GROUP BY customer_id) t1
WHERE rnk = 1

In [ ]:
df = fact_events
result = df.groupby('user_id', as_index=False).agg(cnt=('client_id', 'nunique')).query('cnt == 1')
result['rnk'] = result.sort_values('cnt', ascending=False)['cnt'].rank(method='dense')
df[df['user_id'].isin(result.query('rnk == 1')['user_id'])]['customer_id'].unique()

#### 2031

```Get list of signups which have a transaction start date earlier than 10 months ago from March 2021. For all of those users get the average transaction value and group it by the billing cycle. Your output should include the billing cycle, signup_id of the user, and average transaction amount. Sort your results by billing cycle in reverse alphabetical order and signup_id in ascending order.```

In [ ]:
%%sql
SELECT p.billing_cycle, t.signup_id, AVG(amt) AS avg_amt
FROM transactions t
         JOIN signups s ON t.signup_id = s.signup_id
         JOIN plans p ON p.id = s.plan_id
WHERE t.transaction_start_date < '2021-03-01'::TIMESTAMP - INTERVAL '10 month'
GROUP BY p.billing_cycle, t.signup_id
ORDER BY p.billing_cycle DESC, t.signup_id ASC

In [ ]:
df = pd.merge(pd.merge(transactions, signups, how='inner', on='signup_id'), plans, how='inner', left_on='plan_id',
              right_on='id')
df[(df['transaction_start_date'].dt.date < pd.to_datetime('2021-03-01') - relativedelta(months=10))].groupby(
    ['billing_cycle', 'signup_id'], as_index=False).agg(amt=('amt', 'mean')).sort_values(['amt', 'signup_id'],
                                                                                         ascending=[False, True])

#### ID 2039 

```Find the number of unique transactions and total sales for each of the product categories in 2017. Output the product categories, number of transactions, and total sales in descending order. The sales column represents the total cost the customer paid for the product so no additional calculations need to be done on the column. Only include product categories that have products sold.```

In [ ]:
%%sql
SELECT product_category, COUNT(DISTINCT transaction_id), SUM(sales) AS sales
FROM wfm_transactions AS wt
         JOIN wfm_products AS wp ON wt.product_id = wp.product_id
WHERE EXTRACT(YEAR FROM transaction_date) = 2017
GROUP BY product_category
ORDER BY sales DESC

In [ ]:
df = pd.merge(wfm_transactions, wfm_products, on='product_id', how='inner')
df['year'] = df['transaction_date'].dt.year
df.query('year == 2017').groupby('product_category', as_index=False).agg(cnt=('transaction_id', 'nunique'),
                                                                         sum=('sales', 'sum'))